# Ingest shadow and new acquisition sessions

In [1]:
from scripts.conf_file_finding import try_find_conf_file
try_find_conf_file()

Local configuration file found !!, no need to run the configuration (unless configuration has changed)


## Connection to DB

In [2]:
import utility.display_utility as du
import datajoint as dj
import pandas as pd
import pathlib
import glob

bdata          = dj.create_virtual_module('bdata', 'bl_bdata')
from bl_pipeline.shadow import acquisition as shadow_acquisition
new_acquisition = dj.create_virtual_module('new_acquisition', 'bl_new_acquisition')
new_lab = dj.create_virtual_module('new_lav', 'bl_new_lab')
ratinfo        = dj.create_virtual_module('ratinfo', 'bl_ratinfo')

Connecting alvaros@datajoint01.pni.princeton.edu:3306


## Ingest bl_shadow_acquisition.sessStarted missing sessions

In [3]:
print('Sessions not ingested--> ratname == None')
shadow_acquisition.SessStarted.populate()

Sessions not ingested--> ratname == None
{'sessid': 65694, 'sessiondate': datetime.date(2009, 12, 14), 'starttime': datetime.timedelta(seconds=61510), 'was_ended': 1, 'ratname': None, 'hostname': None, 'technotes': None, 'crashed': 0, 'rigid': 0}
{'sessid': 65695, 'sessiondate': datetime.date(2009, 12, 14), 'starttime': datetime.timedelta(seconds=61514), 'was_ended': 1, 'ratname': None, 'hostname': None, 'technotes': None, 'crashed': 0, 'rigid': 0}
{'sessid': 65698, 'sessiondate': datetime.date(2009, 12, 14), 'starttime': datetime.timedelta(seconds=65773), 'was_ended': 1, 'ratname': None, 'hostname': None, 'technotes': None, 'crashed': 0, 'rigid': 0}
{'sessid': 65699, 'sessiondate': datetime.date(2009, 12, 14), 'starttime': datetime.timedelta(seconds=65772), 'was_ended': 1, 'ratname': None, 'hostname': None, 'technotes': None, 'crashed': 0, 'rigid': 0}
{'sessid': 65700, 'sessiondate': datetime.date(2009, 12, 14), 'starttime': datetime.timedelta(seconds=65778), 'was_ended': 1, 'ratname'

In [4]:
shadow_acquisition.Sessions.populate()

no email, or user_id for experimenter  Satan
sessid  33239  not ingested
no email, or user_id for experimenter  Hessam
sessid  172528  not ingested
no email, or user_id for experimenter  wJeff
sessid  181274  not ingested
no email, or user_id for experimenter  wwwAnn
sessid  185897  not ingested
no email, or user_id for experimenter  Satan
sessid  200949  not ingested
no email, or user_id for experimenter  experimenter
sessid  287226  not ingested
no email, or user_id for experimenter  experimenter
sessid  287227  not ingested
no email, or user_id for experimenter  experimenter
sessid  287228  not ingested
no email, or user_id for experimenter  experimenter
sessid  287229  not ingested
no email, or user_id for experimenter  experimenter
sessid  287271  not ingested
no email, or user_id for experimenter  experimenter
sessid  292412  not ingested
no email, or user_id for experimenter  experimenter
sessid  292637  not ingested
no email, or user_id for experimenter  experimenter
sessid  29

## Ingest bl_new_acquisition.sessStarted missing sessions

In [4]:
#Check missing sessions from shadow
SessStarted = new_acquisition.SessStarted()
all_data = (shadow_acquisition.SessStarted - SessStarted.proj('sessid')).fetch('KEY')
num_sessions_missing = len(all_data)
print('Missing new_acquisition sessStrted:', num_sessions_missing)
for indx, keys in enumerate(all_data):

    du.drawProgressBar(indx/num_sessions_missing)
    data = (bdata.SessStarted & keys).fetch1()

    if data['ratname'] != None:
        entry = dict(
            sessid                   = data['sessid'],
            session_started_rat      = data['ratname'],
            session_started_rigid    = data['rigid'],
            session_date             = data['sessiondate'],
            session_starttime        = data['starttime'],
            was_ended                = data['was_ended'],
            crashed                  = data['crashed']
        )

        if data['rigid'] == 0 and data['hostname'] != None:
            if 'Rig' in data['hostname']:
                    entry.update(session_started_rigid=int(data['hostname'].strip('Rig')))
            else:
                    entry.update(session_started_rigid=int(float(data['hostname'])))
                
        SessStarted.insert1(entry, allow_direct_insert=True, skip_duplicates=True)




Missing new_acquisition sessStrted: 0


## Ingest bl_new_acquisition.sessions missing sessions

In [5]:
#Find missing sessions
Sessions = new_acquisition.Sessions()
all_data = (shadow_acquisition.Sessions * new_acquisition.SessStarted.proj('sessid') - Sessions.proj('sessid')).fetch('KEY')
num_sessions_missing = len(all_data)
print('Missing new_acquisition sessions:', num_sessions_missing)
fields = shadow_acquisition.Sessions.heading.names
for indx, key in enumerate(all_data):

    du.drawProgressBar(indx/num_sessions_missing)
    data = (shadow_acquisition.Sessions & key).fetch(*fields, as_dict=True)[0]
    Sessions.insert1(data, allow_direct_insert=True, skip_duplicates=True)


Missing new_acquisition sessions: 0
